# Feature Engineering for Classification

In [1]:
import pandas as pd

In [2]:
za_data = pd.read_csv("Classification/all_files_ZA.csv")

In [3]:
za_data.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'Position',
       'Track Name', 'Artist', 'Streams', 'Start Date', 'End Date', 'Week',
       'Year', 'Start Date Dt', 'End Date Dt', 'Seconds since Epoch',
       'End Date Aggregated', 'ArtistCount', 'isTopTen', 'lenOnCharts',
       'Artist and Track', 'Position Difference'],
      dtype='object')

In [4]:
len(list(za_data.columns))

19

In [5]:
# groupby_cols = ['Artist', 'Track Name']
groupby_cols = ['Artist']

In [6]:
groupednum_songs = za_data.groupby(groupby_cols)["Track Name"].agg("count")
groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
groupednum_songs_df.columns = groupby_cols + ["Artist Song Count"]

### 1. Streams

In [7]:
groupedmax_streams = za_data.groupby(groupby_cols)["Streams"].agg("max")
groupedmax_streams_df = pd.DataFrame(groupedmax_streams).reset_index()
groupedmax_streams_df.columns = groupby_cols + ["Max Streams per Song"]

In [8]:
groupedmin_streams = za_data.groupby(groupby_cols)["Streams"].agg("min")
groupedmin_streams_df = pd.DataFrame(groupedmin_streams).reset_index()
groupedmin_streams_df.columns = groupby_cols + ["Min Streams per Song"]

In [9]:
groupedsum_streams = za_data.groupby(groupby_cols)["Streams"].agg("sum")
groupedsum_streams_df = pd.DataFrame(groupedsum_streams).reset_index()
groupedsum_streams_df.columns = groupby_cols + ["Total Streams per Song"]

In [10]:
groupedmean_streams = za_data.groupby(groupby_cols)["Streams"].agg("mean")
groupedmean_streams_df = pd.DataFrame(groupedmean_streams).reset_index()
groupedmean_streams_df.columns = groupby_cols + ["Mean Streams per Song"]

### 2. Positions

In [11]:
groupedpos_streams = za_data.groupby(groupby_cols)["Position"].agg("min")
groupedpos_streams_df = pd.DataFrame(groupedpos_streams).reset_index()
groupedpos_streams_df.columns = groupby_cols + ["Min Position per Song"]

groupedposmax_streams = za_data.groupby(groupby_cols)["Position"].agg("max")
groupedposmax_streams_df = pd.DataFrame(groupedposmax_streams).reset_index()
groupedposmax_streams_df.columns = groupby_cols + ["Max Position per Song"]

#### get the dates to use for later other positions

In [12]:
groupeddatemin_streams = za_data.groupby(groupby_cols)['Start Date Dt'].agg("min")
groupeddatemin_streams_df = pd.DataFrame(groupeddatemin_streams).reset_index()
groupeddatemin_streams_df.columns = groupby_cols + ["Min Start Date per Song"]

groupeddatemax_streams = za_data.groupby(groupby_cols)['Start Date Dt'].agg("max")
groupeddatemax_streams_df = pd.DataFrame(groupeddatemax_streams).reset_index()
groupeddatemax_streams_df.columns = groupby_cols + ["Max Start Date per Song"]


groupedenddatemin_streams = za_data.groupby(groupby_cols)['End Date Dt'].agg("min")
groupedenddatemin_streams_df = pd.DataFrame(groupedenddatemin_streams).reset_index()
groupedenddatemin_streams_df.columns = groupby_cols + ["Min End Date per Song"]

groupedenddatemax_streams = za_data.groupby(groupby_cols)['End Date Dt'].agg("max")
groupedenddatemax_streams_df = pd.DataFrame(groupedenddatemax_streams).reset_index()
groupedenddatemax_streams_df.columns = groupby_cols + ["Max End Date per Song"]

### 3. Dates

## Combine the dfs

In [13]:
groupeddf = groupedmax_streams_df

In [14]:
groupeddf["Min Streams per Song"] = groupedmin_streams_df["Min Streams per Song"]
groupeddf["Total Streams per Song"] = groupedsum_streams_df["Total Streams per Song"]
groupeddf["Mean Streams per Song"]= groupedmean_streams_df["Mean Streams per Song"]
groupeddf["Min Position per Song"] = groupedpos_streams_df["Min Position per Song"]
groupeddf["Max Position per Song"] = groupedposmax_streams_df["Max Position per Song"]
groupeddf["Artist Song Count"] = groupednum_songs_df["Artist Song Count"]
# groupeddf["Min Start Date per Song"] = groupeddatemin_streams_df["Min Start Date per Song"]
# groupeddf["Max Start Date per Song"] = groupeddatemax_streams_df["Max Start Date per Song"]
# groupeddf["Min End Date per Song"] = groupedenddatemin_streams_df["Min End Date per Song"]
# groupeddf["Max End Date per Song"] = groupedenddatemax_streams_df["Max End Date per Song"]

In [15]:
groupeddf["MaxMin Difference Position"] = groupeddf["Max Position per Song"] - groupeddf["Min Position per Song"]
groupeddf["MaxMin Difference Streams"] = groupeddf["Max Streams per Song"] - groupeddf["Min Streams per Song"]

In [16]:
groupeddf["MaxMin Ratio Position"] = groupeddf["Max Position per Song"] / groupeddf["Min Position per Song"]
groupeddf["MaxMin Ratio Streams"] = groupeddf["Max Streams per Song"] / groupeddf["Min Streams per Song"]

In [17]:
groupeddf.drop(["Max Position per Song"], inplace=True, axis=1)

In [18]:
# groupeddf["MaxMin Difference Date"] = groupeddf["Min Start Date per Song"] - groupeddf["Max End Date per Song"]

In [19]:
groupeddf.shape

(738, 11)

In [20]:
groupeddf = groupeddf.drop_duplicates()

In [21]:
groupeddf.shape

(738, 11)

In [22]:
groupeddf.to_csv("grouped df.csv", index=False)

In [23]:
from sklearn.cluster import KMeans

In [24]:
kmeans_df = pd.DataFrame()
for col in groupby_cols:
    kmeans_df[col] = groupeddf[col]

In [25]:
train_groupeddf = groupeddf.drop(groupby_cols, axis=1)

In [26]:
# train_groupeddf.head(5)

In [27]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(train_groupeddf)

In [28]:
allklabels = kmeans.labels_

In [29]:
kmeans_df["Labels"] = allklabels

In [32]:
other_cols = ["Min Streams per Song", "Total Streams per Song", "Mean Streams per Song", "Min Position per Song", "Artist Song Count", "MaxMin Ratio Position", "MaxMin Ratio Streams", "MaxMin Difference Position", "MaxMin Difference Streams"]

In [33]:
for col in other_cols:
    kmeans_df[col] = groupeddf[col]

In [34]:
kmeans_df.to_csv("kmeans_df.csv", index=False)

In [ ]:
# df.groupby("date").agg({"duration": np.sum, "user_id": pd.Series.nunique})
